## Image Homogenizer

In [22]:
# Importing useful dependencies
import io
import os
import boto3
from PIL import Image

In [23]:
# Setup S3 client for MinIO (MinIO implements Amazon S3 API)
s3 = boto3.client(
    "s3",
    endpoint_url="http://127.0.0.1:9000", # MinIO API endpoint
    aws_access_key_id="minioadmin", # User name
    aws_secret_access_key="minioadmin", # Password
)

In [24]:
# This function scans a bucket folder, converts every non-PNG image it finds into PNG, uploads it back with the same name but .png extension, and deletes the original file.
def convert_images_to_png(bucket, prefix=""):
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):

            if obj['Size'] == 0: # skip the folder itself
                continue

            key = obj["Key"]

            if os.path.splitext(key)[1].lower() == ".png": # skip png images
                continue

            # New key with .png extension
            new_key = os.path.splitext(key)[0] + ".png"

            # Download the image
            resp = s3.get_object(Bucket=bucket, Key=key)
            body = resp["Body"].read()
            
            # Convert to PNG
            img = Image.open(io.BytesIO(body)).convert("RGBA")
            buf = io.BytesIO()
            img.save(buf, format="PNG")
            buf.seek(0)

            # Upload the image back as PNG (replace original with .png)
            s3.upload_fileobj(buf, Bucket=bucket, Key=new_key, ExtraArgs={"ContentType": "image/png"})

            # Delete the old image
            s3.delete_object(Bucket=bucket, Key=key)

            print(f"Replaced: {key} -> {new_key}")

In [19]:
# Change the format of images to PNG
convert_images_to_png(bucket = "formatted-zone", prefix = "images/")

Replaced: images/image_1759091609544.jpg -> images/image_1759091609544.png
Replaced: images/image_1759091609616.jpg -> images/image_1759091609616.png
Replaced: images/image_1759091609673.jpg -> images/image_1759091609673.png
Replaced: images/image_1759091609731.jpg -> images/image_1759091609731.png
Replaced: images/image_1759091609808.jpg -> images/image_1759091609808.png
Replaced: images/image_1759091609911.jpg -> images/image_1759091609911.png
Replaced: images/image_1759091609988.jpg -> images/image_1759091609988.png
Replaced: images/image_1759091610087.jpg -> images/image_1759091610087.png
Replaced: images/image_1759091610172.jpg -> images/image_1759091610172.png
Replaced: images/image_1759091610248.jpg -> images/image_1759091610248.png
Replaced: images/image_1759091610328.jpg -> images/image_1759091610328.png
Replaced: images/image_1759091610454.jpg -> images/image_1759091610454.png
Replaced: images/image_1759091610514.jpg -> images/image_1759091610514.png
Replaced: images/image_17